In [44]:
# 1:
import paho.mqtt.client as mqtt
import threading
import os
import re
# import motionlocalizer.settings
import json
# import requests
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import xml.etree.ElementTree
import pprint
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.pylab as plt
import matplotlib
from matplotlib.patches import Circle
import time as pythonTimer
import random
from random import randint
import scipy.stats as stats
from scipy.stats import multivariate_normal
import csv
import datetime
import pandas as pd
import datetime
import pandas as pd
import numpy as np

global simworldname
global groundtruth
global motion_readings
global my_map
global grid_map
global distances
global oracle
global analyzer
global dataset_times
global dataset_readings
global dataset
global motions_order
global motion_sensors
global motions_sensor_data
global simulated_sensor_readings
global agentsnum
global sensors_list
global sr_matrix
global indices_to_keep
global gtx
global gty
global gtx2
global gty2
global rsr
global df_
global gtt
global human_readable_sensor_array
global sensed_areas
global plotflag
global syntheticTrace

In [45]:
agentsnum = 2
plotflag = True
sensed_areas = {}
human_readable_sensor_array = []
gtt = []
gtx = []
gty = []
gtx2 = []
gty2 = []
rsr = []
indices_to_keep = [0, 1, 2, 3, 4, 5, 11, 15, 16, 17, 18, 19, 20, 21]
simworldname = 'Data//simulationWorld2.xml' #ENVIROMENT
groundtruth = 'Data//28jun2.csv' #AGENT'S MOVEMENTS
motion_readings = 'Data//28Jun2_motion_events.txt'
syntheticTrace = 'Data//SyntheticAgentTrace.csv'
syntheticTrace2 = 'Data//SyntheticAgentTrace2.csv'
# groundtruth = 'AgentDetailedTasksFile.csv' #AGENT'S MOVEMENTS

my_map = [(0.0, 0.0), (0.0, 10.6), (6.6, 10.6), (6.6, 0.0)]
grid_map = np.zeros((int(my_map[2][0]) * 10, int(my_map[2][1]) * 10))
distances = []

oracle = {}
oracle['time'] = []
oracle['location'] = []

analyzer = {}
analyzer['time'] = []
analyzer['gridmap'] = []
dataset_times = []
dataset_readings = []
dataset = ['time', 'gt(x)', 'gt(y)', 'motion_readings']

motions_order = [1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9]
motion_sensors = [1, 10, 11, 12, 13, 14, 2, 3, 4, 5, 6, 7, 8, 9]
motions_sensor_data = {}
simulated_sensor_readings = []
sensors_list = []
sr_matrix = np.zeros([1, 14])



In [46]:
def SensorReadingInterpretation(dataset_readings):
    templist = [[0] * 14]
    for i in range(0, len(dataset_readings)):
        numlist = [int(s) for s in dataset_readings[i] if s.isdigit()]
        numlist = [numlist[j] for j in range(0, len(numlist)) if j in indices_to_keep]
        templist.append(numlist)
        
    templist_modified = [[0] * 14]
    for row in range(1, len(templist)):
        history = templist_modified[row - 1]
        row_modified = [0] * 14
        for i in range(len(history)):    
            if (templist[row][i] == 0 and history[i] == 1):
                row_modified[i] = 1
   
            elif (templist[row][i] == 0 and history[i] == 0):
                row_modified[i] = 0

            elif (templist[row][i] == 1 and history[i] == 1):
                row_modified[i] = 0

            elif (templist[row][i] == 1 and history[i] == 0):
                row_modified[i] = 1

        templist_modified.append(row_modified)

    del templist_modified[0]
    return templist

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    # print(value - array[idx])
    return idx

In [47]:
# sensor_readings = open(motion_readings, "r")
sensor_readings = open(motion_readings, "r").readlines()

#----- Extracting sensors reading from the groundtruth dataset    
for i in range(0,len(sensor_readings)):
    time = sensor_readings[i].split(';')[0]
    sensor_array = sensor_readings[i].split(';')[1]
    dataset_times.append(int(time))
    dataset_readings.append(sensor_array)

#----- Convert sensor readings to human readable format
# We do not change the values since the job has already been done by Parisa
human_readable_sensor_array = SensorReadingInterpretation(dataset_readings)


In [48]:
temp_gtx = []
temp_gty = []

temp_gtx2 = []
temp_gty2 = []

counter = 0
with open(syntheticTrace, 'r') as agent_data:
    reader = csv.reader(agent_data, delimiter = ',', quotechar = ',')
    line = 0
    for row in reader:
        new_row = []
        temp_gtx.append(6.6 - float(row[2]))
        temp_gty.append(float(row[3]))
        
        if (agentsnum == 2):
            temp_gtx2.append(6.6 - float(row[6]))
            temp_gty2.append(float(row[7]))

        # temp_gtt.append(int(float(row[0])))
        counter += 1

if (agentsnum == 2):
    counter = 0
    with open(syntheticTrace2, 'r') as agent_data:
        reader = csv.reader(agent_data, delimiter = ',', quotechar = ',')
        line = 0
        for row in reader:
            new_row = []
            temp_gtx2.append(6.6 - float(row[6]))
            temp_gty2.append(float(row[7]))

            # temp_gtt.append(int(float(row[0])))
            counter += 1

        
gtx.append(temp_gtx)
gty.append(temp_gty)

if (agentsnum == 2):
    gtx2.append(temp_gtx2)
    gty2.append(temp_gty2)


gtt = dataset_times
gtx = gtx[0]
gty = gty[0]

if (agentsnum == 2):
    gtx2 = gtx2[0]
    gty2 = gty2[0]
    

In [49]:
for i in range(0, len(gtt)):
    ind = find_nearest(dataset_times, int(gtt[i]))
    rsr.append(human_readable_sensor_array[ind])  

In [50]:
if (agentsnum == 2):
    columns = ['time', 'gtx', 'gty', 'gtx2', 'gty2', 'gt_motion_readings', 'synthetic_motion_readings']
    
else:
    columns = ['time', 'gtx', 'gty', 'gt_motion_readings', 'synthetic_motion_readings']

index = range(0, len(gtt))    
df_ = pd.DataFrame(index=index, columns=columns)
    

df_.time = gtt
df_.gtx = gtx[0:len(gtt)]
df_.gty = gty[0:len(gtt)]
df_.gt_motion_readings = rsr

if (agentsnum == 2):
    df_.gtx2 = gtx2[0:len(gtt)]
    df_.gty2 = gty2[0:len(gtt)]

    

In [51]:
df_.to_csv("outputdataset.csv", sep=',', index=False)